In [1]:
import pandas as pd
# this is to avoid getting warnings
pd.options.mode.chained_assignment = None

In [2]:
# load the file to a DataFrame
df = pd.read_excel('/home/macbuse/RECRUTEMENT/MCF_153_LAST.xls')

In [3]:
df.columns

Index(['Campagne', 'Session', 'Corps', 'Section', 'Autre section', 'Article',
       'N° emploi', 'Emplois liés', 'Profil', 'Job profile',
       'Research fields EURAXESS', 'Implant', 'Numéro SI local',
       'Numéro Gesup', 'Localisation', 'Vacance', 'Type de candidature',
       'Chgt de section', 'Civilité', 'Nom', 'Nom d'usage ou marital',
       'Prénom', 'Né(e) le', 'Unnamed: 23', 'Unnamed: 24', 'Rapporteur1',
       'Rapporteur2', 'Avis local', 'Lieu de naissance', 'N° candidat',
       'Référence qualif', 'Numen', 'Etat dossier', 'Détail suivi',
       'Situation professionnelle', 'Lieu d'exercice',
       'Ville ou pays d'exercice', 'Nationalité', 'Adresse1', 'Adresse2',
       'Adresse3', 'Code postal', 'Code étranger', 'Ville', 'Pays',
       'Téléphone', 'Autre tél', 'Fax', 'Email', 'Candidat local', 'Chercheur',
       'Titre thèse', 'Date soutenance', 'Lieu soutenance', 'Directeur Thèse',
       'Jury', 'N° de qualif', 'Activités enseignement', 'Theme', 'motif',
       

---

# Define constants that we'll use later

In [5]:
title = "TEST"

_header_html = f'''<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0       
Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"> 
<html xmlns="http://www.w3.org/1999/xhtml" lang="fr" xml:lang="fr">                                                               
<head> <title>{title}</title>                                         
<meta http-equiv="content-type" content="text/html; charset=UTF-8" />                                                             
'''     


_thesis_fields =  ['Titre thèse', 'Date soutenance', 
                  'Lieu soutenance', 'Directeur Thèse', 
                   u'Jury']                                 
                                                                           
  
_activity_fields = [ u'Situation professionnelle', 
                    u'Activités enseignement',u'Activités administratives']   

_name_fields = ['Nom', 'Prénom', 'Né(e) le']

_web_fields = ['google','arxiv']


In [97]:
candidates[_name_fields]

,Nom,Prénom,Né(e) le
0,ADICEAM,LAKI MICHEL FAUSTIN,20/09/1988
1,ALDANA DOMINGUEZ,CLARA LUCIA,04/12/1976
2,ANNI,SAMUELE,20/06/1985
3,ARANCIBIA ROBERT,NICOLAS JOSE,08/07/1985
4,BALKANOVA,OLGA,27/06/1988
...,...,...,...
103,VENEZIANO,FRANCESCO,06/07/1984
104,VIU SOS,JUAN,04/08/1987
105,VOGLAIRE,YANNICK,06/09/1983
106,ZALAMANSKY,GABRIEL,07/07/1989


---

## Helper functions to generate requests

In [6]:
def fix_name(row):
    ss = "{Prénom} {Nom}".format(**row)
    ss = ss.replace('-',' ')
    return ss.replace(' ','+')

def arxiv(row):
    name = fix_name(row)
    url = f'https://arxiv.org/search/?query={name}'
    url += '&searchtype=all&abstracts=show&order=-announced_date_first&size=50'                  
    request = f'<a href = "{url}" target = "blank"> '    
    request += ' for {Prénom} {Nom} </a>'.format(**row)
    return request

def google(row):
    name = fix_name(row)
    url = f'<a href = "https://www.google.fr/search?q={name}"'
    url += ' target = "blank"> {Prénom} {Nom} </a>'.format(**row)
    return url

---

# Make a web page for all the candidates

In [7]:
candidates = df[~df.iloc[:,0].isnull()]


theses  = candidates[_thesis_fields]
names = candidates[_name_fields]
activity = candidates[_activity_fields]


candidates['google'] = [ google(x) for x in names.to_dict(orient='records') ]
candidates['arxiv'] =  [ arxiv(x) for x in names.to_dict(orient='records') ]

web = candidates[_web_fields]

def table2blocks(dff):
    '''write a dictionnary to rows in html'''
    dico = dff.to_dict(orient='records')
    
    blocks = []
    for row in dico:
        block = [f'<b>{key} : </b> : {value} <br>\n' for key,value in row.items()]
        blocks.append('\n'.join(block) )
    return blocks


blocks = [ table2blocks(_) for _ in [names, web, theses, activity] ]

with open('web.htm','w') as fp:
    fp.write(_header_html)
    for candidate in zip(*blocks):
        fp.write('\n<br>\n'.join(candidate))
        fp.write('<br><hr><br>')
            
            

In [83]:
pwd

'/home/macbuse/TMP'